## Extreme Gradient Boosting

In [4]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

import data_processing
import feature_extraction

In [5]:
df = data_processing.read_all_users('Geolife_data')

[1/38] processing user User37
[2/38] processing user User30
[3/38] processing user User31
[4/38] processing user User36
[5/38] processing user User38
[6/38] processing user User2
[7/38] processing user User5
[8/38] processing user User4
[9/38] processing user User3
[10/38] processing user User25
[11/38] processing user User22
[12/38] processing user User14
[13/38] processing user User13
[14/38] processing user User12
[15/38] processing user User15
[16/38] processing user User23
[17/38] processing user User24
[18/38] processing user User33
[19/38] processing user User34
[20/38] processing user User35
[21/38] processing user User32
[22/38] processing user User8
[23/38] processing user User6
[24/38] processing user User1
[25/38] processing user User7
[26/38] processing user User9
[27/38] processing user User21
[28/38] processing user User26
[29/38] processing user User19
[30/38] processing user User10
[31/38] processing user User17
[32/38] processing user User28
[33/38] processing user Us

In [6]:
df=data_processing.process_classes(df)

In [7]:
df = data_processing.create_segments(df)

In [8]:
df=data_processing.calculations(df)
df

,lat,lon,alt,elapsed time,label,time,trajectory,user,segment,distance,speed,time_diff,acceleration,bearing,heading_change,Vrate,angular_velocity,angular_acceleration
3,39.894600,116.332542,-777.0,39535.623056,train,2008-03-28 14:57:12,01020080328144824.plt,33,1,0.521027,0.008831,59.0,1.851318e-05,94.318545,1.058396e+01,1.236873e-01,1.793892e-01,2.464116e-03
4,39.889622,116.337040,-777.0,39535.623738,train,2008-03-28 14:58:11,01020080328144824.plt,33,1,0.673539,0.011416,59.0,4.381261e-05,145.266086,5.094754e+01,2.264335e-01,8.635176e-01,1.159540e-02
5,39.882090,116.338353,-777.0,39535.624421,train,2008-03-28 14:59:10,01020080328144824.plt,33,1,0.844980,0.014322,59.0,4.925049e-05,172.380803,2.711472e+01,2.028936e-01,4.595715e-01,-6.846546e-03
6,39.873873,116.338455,-777.0,39535.625116,train,2008-03-28 15:00:10,01020080328144824.plt,33,1,0.913730,0.015229,60.0,1.511915e-05,179.454178,7.073376e+00,5.956785e-02,1.178896e-01,-5.694698e-03
7,39.865182,116.338058,-777.0,39535.625799,train,2008-03-28 15:01:09,01020080328144824.plt,33,1,0.966989,0.016390,59.0,1.967467e-05,182.008052,2.553874e+00,7.082555e-02,4.328599e-02,-1.264468e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,40.029320,116.411975,289.0,39781.103808,train,2008-11-29 02:29:29,17920081129015805.plt,36,3851,0.023240,0.011620,2.0,-2.799201e-05,180.419823,2.068969e-01,4.817827e-03,-1.034485e-01,5.147249e-02
472,40.029111,116.411963,275.0,39781.103831,train,2008-11-29 02:29:31,17920081129015805.plt,36,3851,0.023262,0.011631,2.0,5.456289e-06,182.517368,2.097545e+00,9.382246e-04,1.048773e+00,5.761106e-01
473,40.028904,116.411962,274.0,39781.103854,train,2008-11-29 02:29:33,17920081129015805.plt,36,3851,0.023018,0.011509,2.0,-6.117035e-05,180.211944,2.305425e+00,1.063023e-02,-1.152712e+00,-1.100742e+00
474,40.028697,116.411961,274.0,39781.103877,train,2008-11-29 02:29:35,17920081129015805.plt,36,3851,0.023018,0.011509,2.0,-1.147129e-13,180.211944,6.431768e-07,1.993490e-11,3.215884e-07,5.763563e-01


In [9]:
feature_df = df.groupby('trajectory').apply(feature_extraction.extract_features).reset_index(drop=True)

/var/folders/6q/9kdss80974ldrx7hy_83wyn00000gn/T/ipykernel_38553/544797563.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  feature_df = df.groupby('trajectory').apply(feature_extraction.extract_features).reset_index(drop=True)


In [10]:
feature_df

,label,segment,trajectory,start_time,trajectory_length,total_distance,expected_speed,max_speed,mean_speed,top_1_speed,...,duration,average_acceleration,speed_std,acceleration_std,bearing_changes,HCR,stops,SR,VC,VCR
0,train,1,01020080328144824.plt,14,51,61.894997,0.020340,0.031892,60.059836,0.031892,...,1.030556,3.389920e-06,0.011135,0.000059,7,0.137255,3,0.058824,15,0.294118
1,car,2,01020080617094444.plt,12,367,3.459642,0.008484,0.016202,30.676631,0.016202,...,0.112778,-1.364201e-05,0.004158,0.001112,29,0.079019,0,0.000000,52,0.141689
2,train,3,01020080618121033.plt,15,1506,32.994611,0.019507,0.034116,66.506495,0.034116,...,0.496111,2.153045e-05,0.007654,0.000693,8,0.005312,0,0.000000,15,0.009960
3,train,4,01020080618160000.plt,0,9339,263.807927,0.023009,0.034759,70.099538,0.034759,...,3.763333,2.229937e-06,0.009790,0.000530,395,0.042296,343,0.036728,494,0.052896
4,train,5,01020080801023537.plt,15,2177,64.935795,0.026522,0.039892,95.376932,0.039892,...,0.680833,6.046446e-06,0.008630,0.000663,8,0.003675,0,0.000000,19,0.008728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3624,bike,3847,17920081115105653.plt,13,517,2.648993,0.002535,0.014397,9.222799,0.014397,...,0.287222,7.261407e-07,0.001392,0.000624,221,0.427466,2,0.003868,230,0.444874
3625,walk,3848,17920081116065153.plt,7,322,0.897082,0.001393,0.006847,5.030368,0.006847,...,0.178333,-6.023325e-07,0.000796,0.000385,130,0.403727,7,0.021739,162,0.503106
3626,walk,3849,17920081117025429.plt,2,194,0.558652,0.001431,0.005870,5.183372,0.005870,...,0.107778,6.367533e-07,0.000802,0.000421,74,0.381443,2,0.010309,90,0.463918
3627,walk,3850,17920081117030421.plt,7,240,0.723800,0.001495,0.004844,5.317717,0.004844,...,0.136111,9.491422e-07,0.000798,0.000435,101,0.420833,11,0.045833,135,0.562500


Encode labels

In [11]:
encoder = LabelEncoder()
feature_df['label'] = encoder.fit_transform(feature_df['label'])  # Encodes labels into numbers

Train/Test

In [12]:
# Define features and target, drop trajectory column
X = feature_df.drop(columns=['label', 'trajectory'])  # Replace 'label' with actual target column name
y = feature_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=36, stratify=y)
#with straitify: class distribution stays the same on train/test

Deal with class imbalance?

In [25]:
#??

Model

In [13]:
xgb_model = xgb.XGBClassifier(
    objective="multi:softmax", 
    num_class=len(set(y)),  # Set the number of classes for multi-class
    eval_metric="mlogloss",  # Use "logloss" for binary classification
    use_label_encoder=False,
    #scale_pos_weight=scale_pos_weight,  # Adjust weight for imbalance
    n_estimators=100,  # Number of trees
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)






Train the model

In [14]:
# Train model
xgb_model.fit(X_train, y_train)

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:45:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_class=5, num_parallel_tree=None, ...)

Predict the testset

In [15]:
y_pred = xgb_model.predict(X_test)

Evaluate

In [16]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.9173553719008265
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.91      0.91       158
           1       0.83      0.84      0.84        83
           2       0.91      0.90      0.91       124
           3       0.88      0.71      0.79        42
           4       0.94      0.97      0.96       319

    accuracy                           0.92       726
   macro avg       0.90      0.87      0.88       726
weighted avg       0.92      0.92      0.92       726

Confusion Matrix:
 [[143   3   2   0  10]
 [  6  70   2   1   4]
 [  0   7 112   2   3]
 [  1   2   7  30   2]
 [  5   2   0   1 311]]


# Tune Hyperparamters using gridsearch

Set parmaters to search

In [17]:
param_grid = {
    "n_estimators": [100, 200, 300],  # Number of boosting rounds
    "learning_rate": [0.01, 0.1, 0.2],  # Lower values prevent overfitting
    "max_depth": [4, 6, 8],  # Complexity of trees
    "min_child_weight": [1, 3, 5],  # Minimum sum of weights in a child node
    "subsample": [0.8, 1.0],  # Fraction of training samples used
    "colsample_bytree": [0.8, 1.0]  # Fraction of features used per tree
}

In [18]:
xgb_model = xgb.XGBClassifier(
    objective="multi:softmax", 
    num_class=len(set(y)),  # Set the number of classes for multi-class
    eval_metric="mlogloss",  # Use "logloss" for binary classification
    use_label_encoder=False,
    #scale_pos_weight=scale_pos_weight,  # Adjust weight for imbalance
    n_estimators=100,  # Number of trees
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)

In [19]:
# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring="accuracy",
    cv=3,  # 3-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available CPUs
)

In [20]:
# Perform GridSearchCV
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:46:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:46:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:46:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:46:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packa

[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=1.0; total time=   0.6s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=0.8; total time=   0.6s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=1.0; total time=   0.7s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=0.8; total time=   0.7s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=0.8; total time=   0.7s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=1.0; total time=   0.7s


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:46:25] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:46:25] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:46:25] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:46:25] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packa

[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=200, subsample=0.8; total time=   1.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=200, subsample=0.8; total time=   1.4s


...

[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=200, subsample=0.8; total time=   1.4s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=200, subsample=1.0; total time=   1.5s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=200, subsample=1.0; total time=   1.5s


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:50:15] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=200, subsample=1.0; total time=   1.5s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=300, subsample=0.8; total time=   1.7s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=300, subsample=0.8; total time=   1.6s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=300, subsample=0.8; total time=   1.3s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   1.1s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   1.3s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   1.0s


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:50:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='mlogloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=100,
                                     n_jobs=None, num_class=5,
                                     num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [4, 6, 8], 'min_child_weight': [1, 3, 5],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 1.0]},
             scoring='accuracy', verbose=2)

In [21]:
# Print the best parameters
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 300, 'subsample': 0.8}
Best Cross-Validation Accuracy: 0.9111249754288204


In [22]:
best_params = grid_search.best_params_

best_xgb_model = xgb.XGBClassifier(
    objective="multi:softmax",  
    num_class=len(set(y_train)),  
    eval_metric="mlogloss",  
    use_label_encoder=False,
    random_state=42,
    **best_params  # Use best hyperparameters
)

best_xgb_model.fit(X_train, y_train)

# Predictions
y_pred = best_xgb_model.predict(X_test)


# Decode labels back to original transport mode names
y_test = encoder.inverse_transform(y_test)
y_pred = encoder.inverse_transform(y_pred)

# Evaluate model
print("Final Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:02:10] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Final Accuracy: 0.9173553719008265
Classification Report:
               precision    recall  f1-score   support

        bike       0.94      0.90      0.92       158
         bus       0.83      0.87      0.85        83
         car       0.90      0.90      0.90       124
       train       0.88      0.71      0.79        42
        walk       0.94      0.97      0.96       319

    accuracy                           0.92       726
   macro avg       0.90      0.87      0.88       726
weighted avg       0.92      0.92      0.92       726

Confusion Matrix:
 [[142   3   2   0  11]
 [  3  72   3   1   4]
 [  1   7 112   2   2]
 [  0   2   8  30   2]
 [  5   3   0   1 310]]


Use the large dataset

In [35]:
df = data_processing.read_all_users('geolife_trajectories')

[1/182] processing user 135
[2/182] processing user 132
[3/182] processing user 104
[4/182] processing user 103
[5/182] processing user 168
[6/182] processing user 157
[7/182] processing user 150
[8/182] processing user 159
[9/182] processing user 166
[10/182] processing user 161
[11/182] processing user 102
[12/182] processing user 105
[13/182] processing user 133
[14/182] processing user 134
[15/182] processing user 160
[16/182] processing user 158
[17/182] processing user 167
[18/182] processing user 151
[19/182] processing user 169
[20/182] processing user 156
[21/182] processing user 024
[22/182] processing user 023
[23/182] processing user 015
[24/182] processing user 012
[25/182] processing user 079
[26/182] processing user 046
[27/182] processing user 041
[28/182] processing user 048
[29/182] processing user 077
[30/182] processing user 083
[31/182] processing user 084
[32/182] processing user 070
[33/182] processing user 013
[34/182] processing user 014
[35/182] processing use

In [36]:
df = data_processing.process_classes(df)
df = data_processing.drop_unlabelled(df)

In [37]:
df = data_processing.create_segments(df)

In [38]:
df = data_processing.calculations(df)
feature_df = df.groupby('segment').apply(feature_extraction.extract_features).reset_index(drop=True)

/var/folders/6q/9kdss80974ldrx7hy_83wyn00000gn/T/ipykernel_38553/4035211436.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  feature_df = df.groupby('segment').apply(feature_extraction.extract_features).reset_index(drop=True)


In [39]:
feature_df

,label,segment,trajectory,start_time,trajectory_length,total_distance,expected_speed,max_speed,mean_speed,top_1_speed,...,duration,average_acceleration,speed_std,acceleration_std,bearing_changes,HCR,stops,SR,VC,VCR
0,bike,3,20070412102116.plt,10,4,0.068010,0.000373,0.001076,1.050798,0.001076,...,0.064722,0.000006,0.000470,0.000023,3,0.750000,3,0.750000,4,1.000000
1,bike,4,20070412102116.plt,14,44,2.523296,0.002857,0.007365,8.914491,0.007365,...,0.283056,0.000053,0.001183,0.000297,17,0.386364,2,0.045455,17,0.386364
2,bike,6,20070412134621.plt,14,24,2.450483,0.003936,0.004522,14.678434,0.004522,...,0.166944,0.000007,0.000428,0.000028,5,0.208333,0,0.000000,2,0.083333
3,bike,7,20070413005306.plt,0,38,2.051800,0.003347,0.005397,11.487525,0.005397,...,0.178611,0.000008,0.000871,0.000066,21,0.552632,0,0.000000,16,0.421053
4,car,8,20070413005306.plt,5,31,2.124161,0.011816,0.138977,19.408579,0.138977,...,0.109444,0.001153,0.023883,0.013250,16,0.516129,0,0.000000,18,0.580645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9231,bus,18997,20111231075505.plt,7,110,1.471394,0.002690,0.012581,9.719300,0.012581,...,0.151389,0.000017,0.003314,0.000359,61,0.554545,26,0.236364,68,0.618182
9232,walk,18998,20111231075505.plt,8,96,0.927602,0.001664,0.010779,6.071575,0.010779,...,0.152778,0.000014,0.001884,0.000368,59,0.614583,8,0.083333,65,0.677083
9233,walk,18999,20111231144437.plt,14,74,0.505086,0.001297,0.003410,2.734299,0.003410,...,0.184722,0.000010,0.000677,0.000123,41,0.554054,4,0.054054,36,0.486486
9234,car,19000,20111231144437.plt,14,245,14.000611,0.011432,0.021326,41.313277,0.021326,...,0.338889,0.000005,0.006504,0.000319,39,0.159184,20,0.081633,56,0.228571


In [40]:
encoder = LabelEncoder()
feature_df['label'] = encoder.fit_transform(feature_df['label'])  # Encodes labels into numbers

In [41]:
# Define features and target, drop trajectory column
X = feature_df.drop(columns=['label', 'trajectory'])  
y = feature_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=36, stratify=y)

In [42]:
X_train

,segment,start_time,trajectory_length,total_distance,expected_speed,max_speed,mean_speed,top_1_speed,top_2_speed,top_3_speed,...,duration,average_acceleration,speed_std,acceleration_std,bearing_changes,HCR,stops,SR,VC,VCR
127,135,1,170,562.982713,0.031236,0.698681,104.476404,0.698681,0.063781,0.056026,...,5.388611,0.000141,0.053040,0.003545,32,0.188235,4,0.023529,47,0.276471
5751,15028,23,674,14.072890,0.004192,0.027178,15.118593,0.027178,0.018590,0.015329,...,0.930833,0.000008,0.004394,0.000432,275,0.408012,116,0.172107,443,0.657270
6788,16111,1,254,3.130458,0.002107,0.018796,11.813049,0.018796,0.018782,0.018499,...,0.265000,0.000009,0.003252,0.000282,56,0.220472,8,0.031496,47,0.185039
6637,15959,9,568,3.276597,0.002860,0.039595,9.700451,0.039595,0.008847,0.006598,...,0.337778,0.000002,0.001832,0.001205,261,0.459507,5,0.008803,177,0.311620
8520,18254,10,45,0.351555,0.001927,0.010485,5.998096,0.010485,0.005255,0.004728,...,0.058611,0.000136,0.001675,0.000903,29,0.644444,1,0.022222,30,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048,7326,7,4238,285.666827,0.015733,0.033292,65.893546,0.033292,0.033210,0.033131,...,4.335278,0.000001,0.011467,0.000385,230,0.054271,35,0.008259,569,0.134261
5786,15063,11,184,1.209650,0.003231,0.009647,10.832688,0.009647,0.008974,0.008718,...,0.111667,0.000004,0.001760,0.000935,89,0.483696,2,0.010870,113,0.614130
2039,10067,16,410,1.297439,0.001569,0.005334,5.547247,0.005334,0.004776,0.004474,...,0.233889,0.000004,0.000790,0.000444,227,0.553659,3,0.007317,231,0.563415
6060,15340,10,16,2.796103,0.001764,0.005303,16.528686,0.005303,0.002802,0.002702,...,0.169167,0.000043,0.001303,0.000239,9,0.562500,1,0.062500,9,0.562500


Train model on best parameters of the small model

In [43]:
best_xgb_model = xgb.XGBClassifier(
    objective="multi:softmax",  
    num_class=len(set(y_train)),  
    eval_metric="mlogloss",  
    use_label_encoder=False,
    random_state=42,
    **best_params  # Use best hyperparameters
)

best_xgb_model.fit(X_train, y_train)

# Predictions
y_pred = best_xgb_model.predict(X_test)


# Decode labels back to original transport mode names
y_test = encoder.inverse_transform(y_test)
y_pred = encoder.inverse_transform(y_pred)

# Evaluate model
print("Final Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [18:06:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Final Accuracy: 0.8528138528138528
Classification Report:
               precision    recall  f1-score   support

        bike       0.86      0.81      0.83       277
         bus       0.77      0.79      0.78       364
         car       0.86      0.75      0.80       261
       train       0.87      0.81      0.84       154
        walk       0.88      0.94      0.91       792

    accuracy                           0.85      1848
   macro avg       0.85      0.82      0.83      1848
weighted avg       0.85      0.85      0.85      1848

Confusion Matrix:
 [[224  17   1   0  35]
 [ 10 289  24   5  36]
 [  5  38 195  10  13]
 [  0   9   2 124  19]
 [ 21  20   4   3 744]]
